In [14]:
import pickle

from utils.helpers import distance
from default import PROBES_FILE, COUNTRIES_PICKLE_FILE, COUNTRIES_TXT_FILE

from utils.measurement_utils import (
    get_atlas_anchors,
    get_atlas_probes,
    get_atlas_probes_and_anchors,
    load_atlas_anchors,
    load_atlas_probes,
)

## retrieve atlas probes

In [ ]:
# create and fill the probes_and_anchors.pickle dataset.
probes = get_atlas_probes_and_anchors()
print(
    f"selected probes: {len(probes)}")

In [3]:
# create and fill the probes.pickle dataset.
probes, probes_rejected, probes_geoloc_disputed = get_atlas_probes()

# display number of selected and rejected probes.
print(
    f"selected probes: {len(probes)}, rejected: {probes_rejected}, geoloc rejected: {probes_geoloc_disputed} [{round(len(probes) * 100 / (probes_rejected+probes), 4)}%]")

1
1
selected probes: 10156, rejected: 34772, geoloc rejected: 1 [29.207408259519152%]


In [4]:
# create and fill the anchors.pickle dataset.
anchors, anchors_rejected, anchors_geoloc_disputed = get_atlas_anchors()

# display number of selected and rejected anchors.
print(
    f"selected anchors: {len(anchors)}, rejected: {anchors_rejected}, geoloc rejected: {anchors_geoloc_disputed} [{round(len(anchors) * 100 / (anchors_rejected+anchors), 4)}%]")

selected anchors: 767, rejected: 466, geoloc rejected: 1 [164.59227467811158%]


## test loading

In [5]:
probes = load_atlas_probes()
anchors = load_atlas_anchors()
print(f"selected probes: {len(probes)}")
print(f"selected anchors: {len(anchors)}")

selected probes: 10156
selected anchors: 767


## get country goeloc dataset

In [9]:
countries_file = COUNTRIES_TXT_FILE
out_file = COUNTRIES_PICKLE_FILE

countries = {}
with open(countries_file, "r") as f:
    for i, row in enumerate(f.readlines()):

        row = [value.strip() for value in row.split(" ")]
        countries[row[0]] = {
            "latitude": row[1],
            "longitude": row[2],
            "name": row[3],
        }

for i, (country_code, geoloc) in enumerate(countries.items()):
    if i > 10:
        break
    print(f"{country_code} : {geoloc}")

# save results
with open(out_file, "wb") as f:
    pickle.dump(countries, f)

AD : {'latitude': '42.546245', 'longitude': '1.601554', 'name': 'Andorra'}
AE : {'latitude': '23.424076', 'longitude': '53.847818', 'name': 'United'}
AF : {'latitude': '33.93911', 'longitude': '67.709953', 'name': 'Afghanistan'}
AG : {'latitude': '17.060816', 'longitude': '-61.796428', 'name': 'Antigua'}
AI : {'latitude': '18.220554', 'longitude': '-63.068615', 'name': 'Anguilla'}
AL : {'latitude': '41.153332', 'longitude': '20.168331', 'name': 'Albania'}
AM : {'latitude': '40.069099', 'longitude': '45.038189', 'name': 'Armenia'}
AN : {'latitude': '12.226079', 'longitude': '-69.060087', 'name': 'Netherlands'}
AO : {'latitude': '-11.202692', 'longitude': '17.873887', 'name': 'Angola'}
AQ : {'latitude': '-75.250973', 'longitude': '-0.071389', 'name': 'Antarctica'}
AR : {'latitude': '-38.416097', 'longitude': '-63.616672', 'name': 'Argentina'}


## eliminate default geoloc probes

In [10]:
probes_file = PROBES_FILE
countries_file = COUNTRIES_PICKLE_FILE

countries = {}
with open(countries_file, "rb") as f:
    countries = pickle.load(f)

filtered_probes = {}
for probe, probe_description in probes.items():

    # check if probe coordinates are close to default location
    try:
        country_geo = countries[probe_description["country_code"]]
    except KeyError as e:
        print(f"error country code {country_code} is unknown")
        continue

    # if the country code is unknown, remove probe from dataset
    country_lat = float(country_geo["latitude"])
    country_lon = float(country_geo["longitude"])

    probe_lat = float(probe_description["latitude"])
    probe_lon = float(probe_description["longitude"])

    dist = distance(country_lat, probe_lat, country_lon, probe_lon)

    if dist > 5:
        filtered_probes[probe] = probe_description

print(
    f"Number of Atlas probes kept: {len(filtered_probes)}, rejected: {len(probes) - len(filtered_probes)}")

# save results
with open(probes_file, "wb") as f:
    pickle.dump(filtered_probes, f)

error country code AS is unknown
error country code AS is unknown
error country code AS is unknown
error country code AS is unknown
Number of Atlas probes kept: 10082, rejected: 74
